In [7]:
import numpy as np
import pandas as pd
import seaborn as sns

In [8]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [9]:
df.shape

(891, 15)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [11]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [12]:
df['survived'].value_counts()

survived
0    549
1    342
Name: count, dtype: int64

In [13]:
df.groupby('embarked')[['survived']]

In [14]:
df.groupby('embarked')['survived'].mean()

embarked
C    0.553571
Q    0.389610
S    0.336957
Name: survived, dtype: float64

In [40]:
df.groupby('embarked')[['survived']].agg(['sum', 'mean'])

survived          
              sum      mean
embarked                   
C              93  0.553571
Q              30  0.389610
S             217  0.336957

### 위 코드에서 'survived'를 대괄호로 두 번 감싸는 이유
- 첫 번째 대괄호: 판다스에서 열을 선택할 때 사용
- 두 번째 대괄호: 데이터프레임 형태를 유지하기 위해 사용
- 만약 df.groupby('embarked')['survived']처럼 하나의 대괄호만 사용하면 Series가 반환
- 두 개의 대괄호를 사용하면 DataFrame 형태가 유지되어 다중 집계 함수 적용 가능
- 대괄호를 사용하는 것은 "이 데이터의 특정 부분을 선택하겠다"라는 의미이며, 단일 열을 선택하면 Series(또는 SeriesGroupBy), 여러 열을 선택하면 DataFrame(또는 DataFrameGroupBy)으로 출력
- []는 명령어가 아니라 파이썬의 인덱싱 문법을 판다스가 확장해서 사용하는 방법
- 'embarked'를 대괄호로 감싸고, 'survived'에 대괄호를 한 번만 사용해도 동일한 결과 출력

### ```df.groupby('embarked')[['survived']]```를 실행했을 때 바로 DataFrame이 반환되지 않는 이유

groupby() 함수는 바로 계산을 수행하지 않고 그룹화된 데이터에 대해 추가 연산을 적용할 수 있는 중간 객체를 반환. 이것이 'DataFrameGroupBy' 객체
이 객체에 어떤 집계 함수(예: mean(), sum(), count() 등)를 적용해야 최종 결과가 계산됨.

In [16]:
df.groupby('sex')[['survived']].agg(['sum', 'mean'])

survived          
            sum      mean
sex                      
female      233  0.742038
male        109  0.188908

In [17]:
df.groupby('alone')[['survived']].agg(['sum', 'mean'])

survived          
           sum      mean
alone                   
False      179  0.505650
True       163  0.303538

In [18]:
df.groupby(['sex', 'pclass'])['survived'].agg(['sum', 'mean'])

sum      mean
sex    pclass               
female 1        91  0.968085
       2        70  0.921053
       3        72  0.500000
male   1        45  0.368852
       2        17  0.157407
       3        47  0.135447

In [19]:
df.pivot_table(index='sex', columns='pclass', values='survived', aggfunc=['sum', 'mean'])

sum              mean                    
pclass   1   2   3         1         2         3
sex                                             
female  91  70  72  0.968085  0.921053  0.500000
male    45  17  47  0.368852  0.157407  0.135447

### pivot table이란?
- 정의: 데이터 전처리 방법의 일종. 데이터 열 중에서 두 개의 열을 각각 행 인덱스, 열 인덱스를 사용하여 데이터를 조회하고 펼쳐 놓는다.
- 형태
  ```
  pandas.pivot_table(data
                   ## 반드시 알고 있어야 하는 정보
                   , index=None  ## 각 행(row)는 무엇으로 정의할지
                   , columns=None ## 각 열(column)은 무엇으로 정의할지
                   , values=None ## 각 Cell을 어떤 숫자로 계산할지
                   , aggfunc='mean', 'sum', 'nunique' 등 ## 계산을 어떻게 할지
                   
                   ## 더 알고 있으면 좋은 정보
                   , fill_value=None
                   , margins=False
                   , dropna=True
                   , margins_name='All'
                   , observed=False
                   , sort=True
                  )
  ```

In [20]:
df.groupby(['alone', 'sex'])[['survived']].agg(['sum', 'mean'])

# groupby 안에 입력하는 column의 순서에 따라 상위 분류와 하위 분류 다르게 할 수 있음.

survived          
                  sum      mean
alone sex                      
False female      134  0.712766
      male         45  0.271084
True  female       99  0.785714
      male         64  0.155718

In [21]:
df.groupby(['who', 'pclass'])[['survived']].agg(['sum', 'mean'])

# 다수의 column으로 그룹화할 때는 함수 안에 대괄호가 반드시 사용
# 만약 'survived'가 대괄호로 한 번 감싸져 있더라도 결과는 DataFrame 형태로 출력되는데, 이는 pandas가 결과를 sum과 mean 등 다수의 열로 표시해야 하기 때문이다.
# 단, 집계 함수에 하나의 변수만 들어가 있고, 그 안에 대괄호를 사용하지 않는다면 series로 출력된다.

survived          
                  sum      mean
who   pclass                   
child 1             5  0.833333
      2            19  1.000000
      3            25  0.431034
man   1            42  0.352941
      2             8  0.080808
      3            38  0.119122
woman 1            89  0.978022
      2            60  0.909091
      3            56  0.491228

In [22]:
# 인덱스 초기화

result = df.groupby(['who', 'pclass'])['survived'].agg(['sum', 'mean']).reset_index()
result.sort_values('mean', ascending=False, ignore_index=True)

,who,pclass,sum,mean
0,child,2,19,1.000000
1,woman,1,89,0.978022
2,woman,2,60,0.909091
3,child,1,5,0.833333
4,woman,3,56,0.491228
5,child,3,25,0.431034
6,man,1,42,0.352941
7,man,3,38,0.119122
8,man,2,8,0.080808


In [27]:
df.loc[df['who'] == 'child', 'age'].agg(['min', 'max'])

min     0.42
max    15.00
Name: age, dtype: float64

### 위 코드 세부 설명

1. df['who'] == 'child': 'who' 열의 값이 'child'인 행을 찾는다. 이는 True/False 값들로 구성된 series를 생성한다.
2. df.loc[df['who'] == 'child', 'age']: 위의 시리즈를 사용하여 'child'인 행들에서 'age' 열만 선택한다. 이는 아이들의 나이 데이터만 포함하는 Series를 반환한다.
3. .agg(['min', 'max']): 선택된 아이들의 나이 데이터에 대해 최소값('min')과 최대값('max')을 계산한다.

### loc와 iloc 차이
loc와 iloc 모두 판다스에서 데이터를 선택하는 메소드지만 차이점이 있음

***loc(Label-based)***
- 사용 시기: 이름(라벨)으로 데이터에 접근하고자 할 때 사용
- 인덱싱 방식: 인덱스의 이름(라벨)을 기준으로 함.
- 문법: df.loc[행_라벨, 열_라벨]
- 특징
    - 지정한 경계를 포함
    - 불리언 마스크, 라벨 리스트, 단일 라벨 등 다양한 형태로 인덱싱 가능

***iloc(Integer-based)***
- 사용 시기: 위치(정수 인덱스)로 데이터에 접근하고자 할 때 사용
- 인덱싱 방식: 0부터 시작하는 정수 위치를 기준으로 함.
- 문법: df.iloc[행_위치, 열_위치]
- 특징
    - 파이썬의 일반적인 슬라이싱 규칙을 따름(끝 인덱스는 제외).
    - 오직 정수 위치 인덱스만 사용 가능

### `df.loc[[df['who'] == 'child', 'age']].agg(['min', 'max'])`라고 작성하면 오류가 발생하는 이유

.loc는 [행 선택자, 열 선택자] 형태로 사용되어야 함. 두 선택자를 하나의 리스트로 묶어 전달하면 오류 발생(unhashable 타입 에러)
= 두 개의 서로 다른 타입을 포함한 리스트는 .loc의 인덱서로 사용할 수 없음.

### unhashable이란?

***pass***

In [29]:
df.groupby(['pclass', 'who'])[['fare']].mean()

# ~별 수치를 비교할 때는 groupby 사용, 조건이 붙을 때는 loc 사용

fare
pclass who              
1      child  139.382633
       man     65.951086
       woman  104.317995
2      child   28.323905
       man     19.054124
       woman   20.868624
3      child   23.220190
       man     11.340213
       woman   15.354351

In [34]:
rich = df['fare'].quantile(0.9)
print(rich)

# 찾고자 하는 백분위수 값 q는 0과 1 사이의 숫자여야 함.

77.9583


In [33]:
df.loc[df['fare'] >= rich, 'survived'].agg(['count', 'mean'])

count    90.000000
mean      0.766667
Name: survived, dtype: float64

In [50]:
df.groupby(['survived'])['age'].mean()

survived
0    30.626179
1    28.343690
Name: age, dtype: float64

In [51]:
c1 = df['deck'].isnull()
print(df.loc[c1, 'survived']. mean())

0.29941860465116277


In [52]:
c1 = df['deck'].notnull()
print(df.loc[c1, 'survived'].mean())

0.6699507389162561


In [56]:
df.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [63]:
df['embarked'].value_counts()
m = df['embarked'].mode()[0]
df['embarked'] = df['embarked'].fillna(m)

### 위 코드 설명

1. `df['embarked'].value_counts()`
   - `embarked` 열의 값들이 각각 몇 번 등장하는지 세어서 보여준다.
   - 예를 들어, S가 644번, C가 168번, Q가 77번 등의 형태로 결과가 나올 수 있음.
   - 이 명령은 결과를 변수에 저장하지 않고 단순히 출력만 합니다.

2. `m = df['embarked'].mode()[0]`
   - `mode()` 함수를 사용해 `embarked` 열에서 가장 많이 등장하는 값(최빈값)을 찾는다.
   - `mode()`는 여러 개의 최빈값이 있을 경우 모두 반환하므로 `[0]`을 사용해 첫 번째 최빈값만 선택한다.
   - 이 값을 변수 `m`에 저장한다.

3. `df['embarked'] = df['embarked'].fillna(m)`
   - `fillna(m)` 함수를 사용해 `embarked` 열의 모든 결측값(NaN)을 앞서 구한 최빈값 `m`으로 대체한다
   - 결과를 다시 `df['embarked']`에 할당하여 원본 데이터프레임을 업데이트한다.

In [62]:
assert 0 == df['embarked'].isnull().sum()

In [65]:
cond1 = df['sex'] == 'male'
male_mean = df.loc[cond1, 'age'].mean()
df.loc[cond1, 'age'] = df.loc[cond1, 'age'].fillna(male_mean)

cond2 = df['sex'] == 'female'
female_mean = df.loc[cond2, 'age'].mean()
df.loc[cond2, 'age'] = df.loc[cond2, 'age'].fillna(female_mean)

In [66]:
df.groupby('sex', group_keys=False)['age'].mean()

sex
female    27.915709
male      30.726645
Name: age, dtype: float64